<a href="https://colab.research.google.com/github/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/blob/main/Hito_3_Dijkstra_para_Cada_Almacen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import graphviz as gv
import numpy as np
import pandas as pd
import heapq as hq
import time
import math

url = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/almacenesXyY.csv"
url2 = "https://raw.githubusercontent.com/JorgeLMF/cc42_tf_202014828_201915747_201921559_201811355_201714378/main/DATASET/puntosentregaXyY.csv"
file_almacenes = pd.read_csv(url).to_numpy()
pts_entrega = pd.read_csv(url2).to_numpy()

long_per_side=1000

warehouse_points=[]
for line in file_almacenes:
  m=long_per_side*(int(line[0]))+int(line[1])
  warehouse_points.append(m)

delivery_points=[]
for line in pts_entrega:
  m=long_per_side*(int(line[0]))+int(line[1])
  delivery_points.append(m)

In [ ]:
warehouse_points[0]

In [ ]:
delivery_points[0]

In [ ]:
def clusters(warehouse_points, delivery_points): # La funcion clusters pone los puntos de entrega al punto de almacenes más cercano  
  cl = []
  for i in range(len(warehouse_points)):
    cl.append([])

  for j in range(len(delivery_points)):
    d = delivery_points[j]
    mn = math.inf
    idmn = 0
    for i in range(len(warehouse_points)):
      w = warehouse_points[i]
      dis = abs(w - d)
      if(dis < mn):
        mn = dis
        idmn = i
    
    cl[idmn].append(j)
  
  return cl



In [ ]:
cl = clusters(warehouse_points, delivery_points)

In [ ]:
cl[0]

In [ ]:
def djikstra_mod(s, group): # Crea el dijkstra pero la respuesta se basa en el tiempo promedio de la ruta
  n = len(group) + 1
  gr_aux = [s] + group
  
  path = [None]*n
  cost = [math.inf]*n
  cnt = [0]*n
  visited = [False]*n

  cost[0] = 0
  queue = [(0, 0)]
  

  while queue:
    g_u, u = hq.heappop(queue)

    if visited[u]:
      continue
    
    visited[u] = True
    for v in range(len(gr_aux)):
      if(v == u):
        continue

      f = abs(gr_aux[u] - gr_aux[v])

      if f < cost[v] and not visited[v]:
        cost[v] = f
        cnt[v] = cnt[u] + 1
        path[v] = u
        hq.heappush(queue, (f, v))

  mn = math.inf
  idmn = None

  for i in range(1, n): # Ver para cada Dijkstra se mira el tiempo promedio 
    if(cnt[i] == 0):
      continue

    d_prom = cost[i]/cnt[i]

    if(d_prom < mn):
      idmn = i
      mn = d_prom
  

  rec = []

  while(idmn != None):
    rec.append(gr_aux[idmn])
    idmn = path[idmn]

  if(rec == []):
    return [s]

  rec.reverse()


  return rec

In [ ]:
def solve(warehouse_points, clusters): # Crea la ruta de los almacenes
  paths = []
  for i in range(len(warehouse_points)):
    ans = []
    s = warehouse_points[i]

    while(len(clusters[i]) > 0):
      path_sofar = djikstra_mod(s, clusters[i])
      nw_clusters = []

      for x in clusters[i]:
        if(x not in path_sofar):
          nw_clusters.append(x)

      clusters[i] = list(nw_clusters)
      s = path_sofar[-1]
      
      if(len(ans) > 0):
        ans = ans[:-1]
      
      ans+= path_sofar

    print("final de {} : ".format(i), ans)
    paths.append(ans)
  
  return paths

In [ ]:
ps = solve([12,34,56,900], [[10, 11, 34, 21], [28, 32], [48, 50, 52, 5], [980, 921, 878]]) #Faltan los datos